---
title: "Fast AI Dental Radiographs"
description: "Exploring dental radiographs with Fast AI."
author: "Joe Muller"
date: "7/5/2024"
date-modified: "7/5/2024"
image: "image.jpg"
categories: [fastai, deep-learning]
freeze: false
draft: false
---

I am working through the Practical Deep Learning For Coders course from Fast AI and decided to work on my own project for practice. The code below was shared in the [Fast AI Dental Radiographs](https://www.kaggle.com/code/joemuller/fast-ai-dental-radiographs/notebook) notebook on Kaggle. In short, this notebook trains a model to identify cavities, fillings, impacted teeth, and implants in dental radiographs.

# Version 1 - 73.4% Accuracy

Version 1 of the model used a convolutional neural network and had an accuracy of 73.4%.

In [ ]:
import pandas as pd
import os

# List out all CSV input files
input_dir = '/kaggle/input/'

# Initialize lists to store CSV and Excel files
csv_files = []
excel_files = []

# Walk through the input directory and find files
for dirname, _, filenames in os.walk(input_dir):
    for filename in filenames:
        if filename.endswith('.csv'):
            csv_files.append(os.path.join(dirname, filename))
        elif filename.endswith('.xlsx') or filename.endswith('.xls'):
            excel_files.append(os.path.join(dirname, filename))

# Print the list of CSV and Excel files
print("CSV files found:")
for file in csv_files:
    print(file)

print("\nExcel files found:")
for file in excel_files:
    print(file)

#for dirname, _, filenames in os.walk(input_dir):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

In [ ]:
file_path = '/kaggle/input/dental-radiography/train/_annotations.csv'
df = pd.read_csv(file_path)

df.head()

In [ ]:
unique_classes = df['class'].unique()
print("Unique categories in 'class' column:")
print(unique_classes)

In [ ]:
!pip install fastai

In [ ]:
from fastai.vision.all import *

# Filter for implants and fillings
df_filtered = df[df['class'].isin(['Implant', 'Fillings', 'Impacted Tooth', 'Cavity'])]

# Define a function to get image path
def get_image_path(row):
    return f"/kaggle/input/dental-radiography/train/{row['filename']}"

# Define a function to get labels
def get_labels(row):
    return row['class']

# Create a DataBlock
dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_x=get_image_path,
    get_y=get_labels,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    item_tfms=Resize(256)
)

# Create DataLoaders
dls = dblock.dataloaders(df_filtered)
dls.show_batch(max_n=6, figsize=(8, 8))

# Train the model
learn = cnn_learner(dls, resnet34, metrics=accuracy)
learn.fine_tune(4)

# Evaluate the model
learn.show_results()

In [ ]:
learn.export('dental_radiography_model.pkl')

# Attempts at Improvement

With the help of GPT-4o, I made a few attempts at improving the model's performance.

## 1. Changed the model (ex. resnet18, resnet34, resnet50)

The Vision learner supports multiple pretrained models found [here](https://docs.fast.ai/vision.models.xresnet.html).

> `cnn_learner` is a deprecated name for the Fast AI [Vision learner](https://docs.fast.ai/vision.learner.html) so I updated to that in the following experiments.
> 
## 2. [Unfreezed](https://docs.fast.ai/learner.html#learner.unfreeze) the model

## 3. Used [learning rate schedulers](https://d2l.ai/chapter_optimization/lr-scheduler.html)
Instead of updating the algorithm used to train the model, we can update the rate at which the weight vectors of the model are update. I don't understand how this works.

# Deploying the Model

Any model can be made publicly available by following these steps:
1. Create a Hugging Face Space
2. Setup Gradio
3. 

Once deployed to Hugging Face spaces, an API will be generated. You can use this to generate a prediction using the model.